# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,35.62,74,100,11.32,SJ,1698728357
1,1,thompson,55.7435,-97.8558,19.56,73,100,6.91,CA,1698728357
2,2,misratah,32.3754,15.0925,72.00,62,2,14.50,LY,1698728357
3,3,port elizabeth,-33.9180,25.5701,62.31,75,0,13.80,ZA,1698728357
4,4,afaahiti,-17.7500,-149.2833,78.67,83,37,20.69,PF,1698728357


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values

# #A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70)]

# #Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 10)]

# #Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# # Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,al kharijah,25.4514,30.5464,73.62,53,0,6.58,EG,1698728377
111,111,aswan,24.0934,32.9070,74.50,56,0,0.00,EG,1698728382
135,135,kijang,35.2442,129.2139,73.13,41,0,9.91,KR,1698728388
185,185,adrar,20.5022,-10.0711,78.26,14,0,7.49,MR,1698728399
408,408,venice,27.0998,-82.4543,75.22,92,0,6.91,US,1698728507
416,416,zouerate,22.7187,-12.4521,77.09,19,0,5.46,MR,1698728509
557,557,shenjiamen,29.9576,122.2980,72.16,58,0,7.05,CN,1698728557
576,576,sultanah,24.4926,39.5857,72.05,68,0,5.75,SA,1698728562


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
90,al kharijah,EG,25.4514,30.5464,53,
111,aswan,EG,24.0934,32.9070,56,
135,kijang,KR,35.2442,129.2139,41,
185,adrar,MR,20.5022,-10.0711,14,
408,venice,US,27.0998,-82.4543,92,
416,zouerate,MR,22.7187,-12.4521,19,
557,shenjiamen,CN,29.9576,122.2980,58,
576,sultanah,SA,24.4926,39.5857,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key  # Make sure geoapify_key contains your actual API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al kharijah - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
kijang - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
venice - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
shenjiamen - nearest hotel: No hotel found
sultanah - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
90,90,al kharijah,25.4514,30.5464,73.62,53,0,6.58,EG,1698728377,No hotel found
111,111,aswan,24.0934,32.9070,74.50,56,0,0.00,EG,1698728382,No hotel found
135,135,kijang,35.2442,129.2139,73.13,41,0,9.91,KR,1698728388,No hotel found
185,185,adrar,20.5022,-10.0711,78.26,14,0,7.49,MR,1698728399,No hotel found
408,408,venice,27.0998,-82.4543,75.22,92,0,6.91,US,1698728507,No hotel found
416,416,zouerate,22.7187,-12.4521,77.09,19,0,5.46,MR,1698728509,No hotel found
557,557,shenjiamen,29.9576,122.2980,72.16,58,0,7.05,CN,1698728557,No hotel found
576,576,sultanah,24.4926,39.5857,72.05,68,0,5.75,SA,1698728562,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)